# Ler dados das estações automaticas do INMET

v1.1 implementado a busca das tabelas no banco de dados.

In [1]:
#importar as bibliotecas
import pandas as pd
import sqlite3 as sql
import datetime as dt
import time

# Buscar a situação das estacaoes automaticas


# TODO: implementar a leitura do codigo da estacao a partir do arquivo CSV

Link tutorial json with pans
https://stackabuse.com/reading-and-writing-json-files-in-python-with-pandas/#:~:text=To%20read%20a%20JSON%20file,form%20of%20columns%20and%20rows.

# Ler dados a partir do banco de dados pynmet

link tutorial sqlite
https://towardsdatascience.com/python-pandas-and-sqlite-a0e2c052456f

# Parte do código que deve ser executada uma única vez... 

In [1676]:
# codigo da estacao que desejamos atualizar
#codigo="A803"
# baixar todas as tabelas
conn = sql.connect("inmet.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# retorna uma lista com todas as estacoes armazenadas
tabelas = cursor.fetchall()
#fechar a conexao com o banco de dados
cursor.close()
#cria o index de controle
navegacao = 0

In [3]:
#Organiza uma lista para iniiar as atualizacoes om a API do INMET
lista = []
for i in range(0,len(tabelas)):
    #print (tabelas[i][0])
    lista.append(tabelas[i][0])

In [4]:
#ordenar a lista
lista.sort()

# Parte do codigo responsavel por atualizar...
# Sempre voltar aqui para atualizar a próxima estacão

In [2280]:
# baixar todas as datas

codigo=lista[navegacao]
conn = sql.connect("inmet.db")
query = """
SELECT TIME FROM {codigo}
""".format(codigo=codigo)
db = pd.read_sql(query,conn)

# Buscar a data do ultimo registro cadastrado no banco de dados

In [2281]:
#definir TIME como indice
db.set_index('TIME', inplace=True)

In [2282]:
if len(db) == 0:
    print('Tabela para a estacao {codigo} nao existe ou está vazia'.format(codigo=codigo))
else:
    print('Tabela para a estacao {codigo} foi encontrada com sucesso'.format(codigo=codigo))

Tabela para a estacao A301 foi encontrada com sucesso


# Checar registros de datas duplicadas

In [2283]:
if db[db.index.duplicated()].empty:
    print('Tudo  em ordem na estacao {codigo}'.format(codigo=codigo))
else:
    print('Foram encontrados dados duplicados. Implementar solucao!!')

Tudo  em ordem na estacao A301


In [2284]:
# formato de data para buscar na API do INMET
fmt = "%Y-%m-%d"

# ultimo registro cadastrdo da estacao
data_u = db.index.max().split(' ')[0]

#converte data do formato texto para o tipo datetime. Util para fazer operacoes de calculo com  data
data_u = dt.datetime(int(data_u.split('-')[0]),int(data_u.split('-')[1]),int(data_u.split('-')[2]))

print(data_u.strftime(fmt))


2021-05-24


# buscar um periodo entre a ultimo registro e o dia de hoje
# * até o momento não existe um limitação quanto ao número de dias

# definir alguns limites para o periodo.
Não deve exceder a data final não deve exceder o dia de hoje

In [2285]:
data_h = dt.datetime.now()
periodo = data_h - data_u
print('Ultima data registrada: {data}'.format(data = data_u.strftime(fmt)))

if (data_h.strftime(fmt) == (data_u.strftime(fmt))):
    print('Estacao {codigo} esta atualizada'.format(codigo = codigo))
    atualizar = False
else:
    print("intervalo de busca de {dias} dias".format(dias=periodo.days))
    atualizar = True

Ultima data registrada: 2021-05-24
intervalo de busca de 115 dias


In [2286]:
data_f = data_u + dt.timedelta(days=periodo.days)
print(data_f.strftime(fmt))

2021-09-16


In [2287]:
data_i = data_u.strftime(fmt)
data_f = data_f.strftime(fmt)

url = "https://apitempo.inmet.gov.br/estacao/"+data_i+"/"+data_f+"/"+codigo

if atualizar:
    print(url)
else:
    print('Tudo esta em ordem com a estacao {codigo}'.format(codigo=codigo))

https://apitempo.inmet.gov.br/estacao/2021-05-24/2021-09-16/A301


# realizar a consulta pela API e criar o dateframe com o resultdo

In [2288]:
if atualizar:
    df = pd.read_json(url)
else:
    print('Nada a fazer. A estacao {codigo} esta atualizada'.format(codigo=codigo))

# Seguir os passos do 1 ate o 8 caso seja necessário atualizar.

# 1. Renomear as colunas compativel com o padrão do Pynmet
(https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas)

In [2289]:
df = df.rename(columns = {'TEM_INS':'Temperatura', 'TEM_MAX':'Temperatura_max', 'TEM_MIN':'Temperatura_min', 'UMD_INS':'Umidade',
          'UMD_MAX':'Umidade_max', 'UMD_MIN':'Umidade_min', 'PTO_INS':'Ponto_orvalho',
          'PTO_MAX':'Ponto_orvalho_max', 'PTO_MIN':'Ponto_orvalho_min', 'PRE_INS':'Pressao',
          'PRE_MAX':'Pressao_max', 'PRE_MIN':'Pressao_min', 'VEN_VEL':'Vento_velocidade', 'VEN_DIR':'Vento_direcao',
          'VEN_RAJ':'Vento_rajada', 'RAD_GLO':'Radiacao', 'CHUVA':'Precipitacao'})

# 2. Criar datetime [2019/05/29 13:00:00 ] com as colunas 'DT_MEDICAO' e 'HR_MEDICAO'  


# 3. Definir o datetime criado como index do dataframe

In [2290]:
# ajuste da hora para ficar no padrao %H Ex 1 ao inves de 100
df['HR_MEDICAO'] = df['HR_MEDICAO'].apply(lambda x: x/100).astype(int)
#converte a data e hora para o tipo texto
df[['DT_MEDICAO','HR_MEDICAO']] = df[['DT_MEDICAO','HR_MEDICAO']].astype(str)
#Cria uma lista com o datetime no paadrao %Y-%m-%d%H 2020-05-01 13:00:00
data = pd.to_datetime(df['DT_MEDICAO'] + df['HR_MEDICAO'], format="%Y-%m-%d%H")
#Associa o datetime ao indice dos dados baixados
df.set_index(data, inplace=True)

# 4. Apagar colunas que não precisamos 
['DT_MEDICAO','DC_NOME','VL_LATITUDE','UF','CD_ESTACAO','VL_LONGITUDE','HR_MEDICAO']

In [2291]:
df.drop(['DT_MEDICAO','DC_NOME','VL_LATITUDE','UF','CD_ESTACAO','VL_LONGITUDE','HR_MEDICAO'], axis=1, inplace=True)

# 5. Reordenar as colunas na sequencia armazenada no inmet.db
(https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns)

In [2292]:
df = df[['Temperatura', 'Temperatura_max', 'Temperatura_min', 'Umidade', 'Umidade_max', 'Umidade_min', 'Ponto_orvalho', 'Ponto_orvalho_max', 'Ponto_orvalho_min', 'Pressao', 'Pressao_max', 'Pressao_min', 'Vento_velocidade', 'Vento_direcao', 'Vento_rajada', 'Radiacao', 'Precipitacao']]

# 6. verificar se o periodo baixado do INMET já está cadastrado no banco de dados do pynmet

# 7. Remover linhas caso todos os valores sejam NA

In [2293]:
# se no banco de dados a ultima informacao gravada foi a leitura das 13h do dia 20/05/2020,
# então apenas devera mostrar as leituras a partir das 14h em diante do dia 20/05/2020.
# evitar duplicar informações salvas no db
dados = df[~df.index.isin(db.index)]

In [2294]:
# elimina as linhas da data de hj que não tem cadastro no INMET.
dados = dados.dropna(how='all')

# Gravar as informações no banco de dados
# Backup do arquivo antes de fazer alterações
# Revisar os dados antes de salvar!!!!

In [2295]:
if len(dados) ==0:
    gravar = False
    print ('A estacao esta inoperante durante o periodo solicitado: {data1} - {data2}'.format(data1=data_i,data2=data_f))
else:
    gravar= True

dados.head(-5)

,Temperatura,Temperatura_max,Temperatura_min,Umidade,Umidade_max,Umidade_min,Ponto_orvalho,Ponto_orvalho_max,Ponto_orvalho_min,Pressao,Pressao_max,Pressao_min,Vento_velocidade,Vento_direcao,Vento_rajada,Radiacao,Precipitacao
2021-05-24 16:00:00,30.3,30.5,28.2,59.0,69.0,59.0,21.4,22.4,20.4,1012.3,1013.1,1012.3,2.2,115.0,6.4,2320.934,0.0
2021-05-24 17:00:00,29.2,30.3,28.5,63.0,66.0,58.0,21.5,21.6,20.0,1011.6,1012.3,1011.6,2.1,121.0,5.9,1730.853,0.0
2021-05-24 18:00:00,29.1,30.0,29.1,61.0,63.0,58.0,20.9,21.8,20.6,1011.6,1011.6,1011.5,2.2,114.0,6.0,2171.761,0.0
2021-05-24 19:00:00,28.4,29.2,27.8,66.0,68.0,60.0,21.4,21.8,20.5,1011.6,1011.7,1011.5,1.7,110.0,6.2,1267.740,0.0
2021-05-24 20:00:00,26.7,28.4,26.7,73.0,73.0,66.0,21.4,21.5,21.1,1011.8,1011.9,1011.6,1.7,115.0,5.4,226.405,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-15 08:00:00,23.6,23.7,23.5,99.0,100.0,99.0,23.5,23.7,23.4,1014.3,1014.3,1013.8,0.9,131.0,3.4,-2.574,0.0
2021-09-15 09:00:00,24.3,24.3,23.5,99.0,100.0,99.0,24.1,24.1,23.4,1014.8,1014.8,1014.3,1.7,127.0,3.7,86.410,0.0
2021-09-15 10:00:00,25.9,25.9,24.3,92.0,99.0,92.0,24.5,24.8,24.1,1015.4,1015.4,1014.8,2.1,115.0,5.4,538.194,0.0
2021-09-15 11:00:00,27.6,27.7,25.7,73.0,92.0,73.0,22.3,24.2,22.3,1015.9,1015.9,1015.4,2.4,120.0,7.0,1645.910,0.0


# 8. Atualizar o banco de dados com as novas informações

In [2296]:
if gravar:
    dados.to_sql(codigo, conn, if_exists='append', index_label='TIME')
    print('{tempo} -> Dados da estacao {navegacao}:{codigo} atualizada com sucesso!'.format(codigo=codigo,navegacao=navegacao,tempo=dt.datetime.now()))
    #fechar a conexao com banco de dados
    conn.close()
    #time.sleep(10)
    navegacao = navegacao + 1
else:
    print('{tempo} -> Não foram feitas alterações na estacao {navegacao}:{codigo}.'.format(codigo=codigo,navegacao=navegacao,tempo=dt.datetime.now()))
    #fechar a conexao com banco de dados
    conn.close()
    #time.sleep(10)
    navegacao = navegacao + 1

2021-09-16 16:52:09.860235 -> Dados da estacao 134:A301 atualizada com sucesso!
